The devset of the MS MARCO passage ranking dataset it fully given with the qrels.dev.tsv file (55578 queries). The top1000.dev file is reduced for practicability reasons (6980 queries).
This notebook takes the qrels and top1000.dev as inputs and outputs a new tsv file with in the format (query string, passage text, relevance label) for the top1000.dev entries we also have qrels for.
Additionally, we sample an equal size of query-passage pairs from top1000.dev for which we do not have qrels and use those as negative examples.
The final dataset is then written to file.

In [1]:
import pandas as pd

In [19]:
top1000dev = pd.read_csv("top1000.dev.gz", sep="\t", names=["qid", "pid", "q_text", "p_text"]).set_index(["qid", "pid"])

In [45]:
qrels = pd.read_csv("qrels.dev.small.tsv", sep="\t", names=["qid", "Q0", "pid", "relevance"]).set_index(["qid", "pid"])

We use the full qrels to filter the top1000dev passages to make sure we only sample negative example that we really don't have qrels for (note that the MS MARCO qrels only list relevant passages)

In [110]:
qrels_full = pd.read_csv("qrels.dev.tsv", sep="\t", names=["qid", "Q0", "pid", "relevance"]).set_index(["qid", "pid"])

In [72]:
top1000dev_qrels = qrels.join(top1000dev, how="inner")

In [111]:
top1000dev_noqrels = top1000dev[qrels_full.join(top1000dev, how="right").isnull().any(axis=1)]

In [112]:
top1000dev_negative_examples = top1000dev_noqrels.sample(len(top1000dev_qrels), random_state=4)

In [113]:
negative_examples = top1000dev_negative_examples.join(qrels)
negative_examples["Q0"] = 0
negative_examples["relevance"] = 0

In [114]:
positive_examples = top1000dev.join(qrels, how="inner")

In [122]:
dev_all = pd.concat([positive_examples, negative_examples])

In [123]:
dev_all.to_csv("mydevset.tsv", sep="\t")